In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
aq=pd.read_hdf("air_quality_data.h5",key ='data')

In [7]:
from data_transformation import obtener_registros_por_hora
from scipy.interpolate import griddata
def generate_interpolations(dataframe):
    fecha_hora_unicas = dataframe['fecha'].unique()
    interpolations = []

    for date_time in fecha_hora_unicas:
        df_hora = obtener_registros_por_hora(dataframe, date_time)

        lat = df_hora['latitud']
        lon = df_hora['longitud']
        aqi = df_hora['aqi']

        longitud_grid = np.linspace(lon.min(), lon.max(), 100)
        latitud_grid = np.linspace(lat.min(), lat.max(), 100)
        longitud_mesh, latitud_mesh = np.meshgrid(longitud_grid, latitud_grid)

        grid_interpolado_cubico = griddata((lon, lat), aqi, (longitud_mesh, latitud_mesh), method='cubic')
        puntos_faltantes = np.isnan(grid_interpolado_cubico)
        grid_interpolado_vecinos = griddata((lon, lat), aqi, (longitud_mesh, latitud_mesh), method='nearest')
        grid_interpolado_combinado = np.where(puntos_faltantes, grid_interpolado_vecinos, grid_interpolado_cubico)

        interpolations.append(grid_interpolado_combinado)

    return interpolations


c:\Users\FELIPE.LAPTOP-20LUI4GA\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [8]:
interpolations = generate_interpolations(aq)

In [28]:
primera_interpolacion = interpolations[0]

# Grafica la interpolación
plt.figure(figsize=(12, 8))
longitud_mesh, latitud_mesh = np.meshgrid(longitud_grid, latitud_grid)
plt.contourf(latitud_mesh, longitud_mesh, primera_interpolacion, cmap='YlOrRd')
plt.xlabel('Latitud')
plt.ylabel('Longitud')
plt.title('Interpolación del Primer Elemento de la Lista')
plt.colorbar(label='AQI')
plt.show()


135.74986432160804

In [14]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from data_transformation import obtener_registros_por_hora

def generate_interpolation_dataframe(dataframe):
    fecha_hora_unicas = dataframe['fecha'].unique()
    interpolations = []

    for date_time in fecha_hora_unicas:
        df_hora = obtener_registros_por_hora(dataframe, date_time)

        lat = df_hora['latitud']
        lon = df_hora['longitud']
        aqi = df_hora['aqi']

        longitud_grid = np.linspace(lon.min(), lon.max(), 100)
        latitud_grid = np.linspace(lat.min(), lat.max(), 100)
        longitud_mesh, latitud_mesh = np.meshgrid(longitud_grid, latitud_grid)

        grid_interpolado_cubico = griddata((lon, lat), aqi, (longitud_mesh, latitud_mesh), method='cubic')
        puntos_faltantes = np.isnan(grid_interpolado_cubico)
        grid_interpolado_vecinos = griddata((lon, lat), aqi, (longitud_mesh, latitud_mesh), method='nearest')
        grid_interpolado_combinado = np.where(puntos_faltantes, grid_interpolado_vecinos, grid_interpolado_cubico)

        interpolations.append(grid_interpolado_combinado)

    interpolation_df = pd.DataFrame({
        "fecha": fecha_hora_unicas,
        "interpolacion": interpolations
    })

    return interpolation_df


In [29]:
interpolations_df=generate_interpolation_dataframe(aq)

In [33]:
interpolations_df

,fecha,interpolacion
0,2019-02-21 04:00:00,"[[135.74986432160804, 135.74986432160804, 135...."
1,2019-02-21 05:00:00,"[[152.92845205479452, 152.92845205479452, 152...."
2,2019-02-21 06:00:00,"[[120.63176381909548, 120.63176381909548, 120...."
3,2019-02-21 07:00:00,"[[138.28087437185928, 138.28087437185928, 138...."
4,2019-02-21 08:00:00,"[[132.7123567839196, 132.7123567839196, 132.71..."
...,...,...
8756,2019-11-26 14:00:00,"[[47.96875, 47.96875, 47.96875, 47.96875, 47.9..."
8757,2019-07-01 23:00:00,"[[40.33675, 40.33675, 40.33675, 40.33675, 40.3..."
8758,2019-11-16 08:00:00,"[[95.81228755364808, 95.81228755364808, 95.812..."
8759,2019-11-16 09:00:00,"[[102.81103015075378, 102.81103015075378, 102...."


In [34]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata

def generate_daily_interpolation_average_dataframe(dataframe):
    # Crear una lista para almacenar los datos de salida
    daily_averages = []

    # Obtener la lista única de fechas
    unique_dates = dataframe['fecha'].dt.date.unique()

    for date in unique_dates:
        # Filtrar el DataFrame por fecha
        daily_data = dataframe[dataframe['fecha'].dt.date == date]

        # Coordenadas de latitud y longitud
        latitudes = daily_data['latitud']
        longitudes = daily_data['longitud']

        # AQI para el día
        aqi = daily_data['aqi']

        # Crear una cuadrícula de coordenadas
        lat_grid = np.linspace(latitudes.min(), latitudes.max(), 100)
        lon_grid = np.linspace(longitudes.min(), longitudes.max(), 100)
        lat_mesh, lon_mesh = np.meshgrid(lat_grid, lon_grid)

        # Interpolación cúbica
        interpolated_values = griddata((latitudes, longitudes), aqi, (lat_mesh, lon_mesh), method='cubic')

        # Calcular el promedio de la interpolación para el día
        daily_average = interpolated_values.mean()

        # Almacenar la fecha y el promedio en la lista de resultados
        daily_averages.append([date, daily_average])

    # Crear un DataFrame a partir de la lista de resultados
    daily_average_df = pd.DataFrame(daily_averages, columns=['fecha', 'promedio_interpolacion'])

    return daily_average_df

# Ejemplo de uso:
# df_daily_averages = generate_daily_interpolation_average_dataframe(air_quality_data)


In [ ]:
df_daily_averages = generate_daily_interpolation_average_dataframe(aq)

In [ ]:
filtro = aq['aqi'] > 100.0
aq_bad = aq[filtro]